In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from surprise import Reader, Dataset, SVD
import time
from surprise.accuracy import rmse, mae
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

In [22]:
### read ratings.csv file
ratings = pd.read_csv("./BX-Book-Ratings.csv",sep=';', encoding='latin-1',error_bad_lines=False)
ratings.head()

C:\Users\nguoi\AppData\Local\Temp\ipykernel_13332\2448960088.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings = pd.read_csv("./BX-Book-Ratings.csv",sep=';', encoding='latin-1',error_bad_lines=False)


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [42]:
books = pd.read_csv("./BX-Books.csv",sep=';', encoding='latin-1',error_bad_lines=False)
books.head()

C:\Users\nguoi\AppData\Local\Temp\ipykernel_13332\2826857486.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv("./BX-Books.csv",sep=';', encoding='latin-1',error_bad_lines=False)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957:

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [84]:
users = pd.read_csv("./BX-Users.csv",sep=';', encoding='latin-1',error_bad_lines=False)
users.head()

C:\Users\nguoi\AppData\Local\Temp\ipykernel_13332\357173605.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users = pd.read_csv("./BX-Users.csv",sep=';', encoding='latin-1',error_bad_lines=False)


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


Kích thước của tập dữ liệu ratings ban đầu

In [23]:
ratings.shape

(1149780, 3)

In [24]:
filter_non_ratings = ratings[ratings['Book-Rating'] != 0 ]

Kích thước của tập dữ liệu sau khi loại bỏ các ratings không có

In [25]:
filter_non_ratings.shape

(433671, 3)

In [52]:
books_with_ratings = filter_non_ratings.join(books.set_index('ISBN'), on='ISBN')
print(f'New table size: {len(books_with_ratings)}')

New table size: 433671


In [54]:
### kiểm tra số sánh không có tác giả tiêu đề
print(f'Co {books_with_ratings["Book-Title"].isnull().sum()} sach khong co tac gia tieu de.')
print(f'co {len(books_with_ratings)/books_with_ratings["Book-Title"].isnull().sum():.2f}% cua tap du lieu.')

Co 49829 sach khong co tac gia tieu de.
co 8.70% cua tap du lieu.


In [55]:
books_with_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433671 entries, 1 to 1149779
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   User-ID              433671 non-null  int64 
 1   ISBN                 433671 non-null  object
 2   Book-Rating          433671 non-null  int64 
 3   Book-Title           383842 non-null  object
 4   Book-Author          383841 non-null  object
 5   Year-Of-Publication  383842 non-null  object
 6   Publisher            383840 non-null  object
 7   Image-URL-S          383842 non-null  object
 8   Image-URL-M          383842 non-null  object
 9   Image-URL-L          383841 non-null  object
dtypes: int64(2), object(8)
memory usage: 36.4+ MB


In [56]:
## Loại bỏ các giá trị Na của cột book-title
books_with_ratings.dropna(subset=["Book-Title"], inplace=True) 

In [59]:
### Kiểm tra xem 1 cuốn sách có nhiều mã sách hay không
books_with_ratings.groupby("Book-Title")["ISBN"].nunique().sort_values(ascending=False)[:10]


Book-Title
The Secret Garden                 18
Selected Poems                    16
Dracula                           15
Adventures of Huckleberry Finn    15
Pride and Prejudice               14
Beloved                           14
Little Women                      14
Jane Eyre                         14
Wuthering Heights                 14
The Hobbit                        12
Name: ISBN, dtype: int64

In [62]:
multiple_isbns = books_with_ratings.groupby("Book-Title")['ISBN'].nunique()
multiple_isbns.value_counts()

1     125336
2       7872
3       1497
4        493
5        181
6         86
7         43
8         28
9         11
10         8
14         5
15         2
11         2
16         1
12         1
18         1
Name: ISBN, dtype: int64

In [63]:
## Loại bỏ các giá trị na trong trường là những cuốn sách chỉ có 1 mã sách
has_mult_isbns = multiple_isbns.where(multiple_isbns>1)
has_mult_isbns.dropna(inplace=True) # remove NaNs, which in this case is books with a single ISBN number

In [79]:
def add_unique_isbn_col(df):
    df['unique_isbn'] = df.apply(lambda row: multiple_isbn_dict[row["Book-Title"]][0] if row["Book-Title"] in multiple_isbn_dict.keys() else row["ISBN"], axis=1)
    return df


In [80]:
def make_isbn_dict(df):
     title_isbn_dict = {}
     for title in has_mult_isbns.index:
         isbn_series = df.loc[df["Book-Title"]==title]["ISBN"].unique() # returns only the unique ISBNs
         title_isbn_dict[title] = isbn_series.tolist()
     return title_isbn_dict

In [71]:
dict_unique_isbn = make_isbn_dict(books_with_ratings)

In [76]:
with open('multiple_isbn_dict.pickle', 'wb') as handle:
     pickle.dump(dict_unique_isbn, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [77]:
with open('multiple_isbn_dict.pickle', 'rb') as handle:
    multiple_isbn_dict = pickle.load(handle)

In [81]:
books_with_ratings = add_unique_isbn_col(books_with_ratings)

In [83]:
### Xêm thử 1 cuốn sách với tên là Jane Eyre có nhiều mã sách hay không.
books_with_ratings[books_with_ratings["Book-Title"]=='Jane Eyre'].head()


,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,unique_isbn
10481,387,1590071212,9,Jane Eyre,Charlotte Bronte,2002,New Millenium Audio,http://images.amazon.com/images/P/1590071212.0...,http://images.amazon.com/images/P/1590071212.0...,http://images.amazon.com/images/P/1590071212.0...,1590071212
16178,2718,0451518845,10,Jane Eyre,Charlotte Bronte,1999,New Amer Library Classics,http://images.amazon.com/images/P/0451518845.0...,http://images.amazon.com/images/P/0451518845.0...,http://images.amazon.com/images/P/0451518845.0...,1590071212
35499,8370,0451523326,8,Jane Eyre,Charlotte Bronte,1988,Signet Classics,http://images.amazon.com/images/P/0451523326.0...,http://images.amazon.com/images/P/0451523326.0...,http://images.amazon.com/images/P/0451523326.0...,1590071212
50842,11676,0451523326,9,Jane Eyre,Charlotte Bronte,1988,Signet Classics,http://images.amazon.com/images/P/0451523326.0...,http://images.amazon.com/images/P/0451523326.0...,http://images.amazon.com/images/P/0451523326.0...,1590071212
58066,11676,843761449X,8,Jane Eyre,Charlotte Bronte,1998,Ediciones Catedra S.A.,http://images.amazon.com/images/P/843761449X.0...,http://images.amazon.com/images/P/843761449X.0...,http://images.amazon.com/images/P/843761449X.0...,1590071212


In [85]:
### Kiểm tra kích thước của tập dữ liệu
print(f'Books+Ratings table size: {len(books_with_ratings)}')
print(f'Users table size: {len(users)}')
books_users_ratings = books_with_ratings.join(users.set_index('User-ID'), on='User-ID')
print(f'New "books_users_ratings" table size: {len(books_users_ratings)}')

Books+Ratings table size: 383842
Users table size: 278858
New "books_users_ratings" table size: 383842


In [86]:
books_users_ratings.shape


(383842, 13)

In [87]:
books_users_ratings.to_csv("New-Ratings.csv", index= False)

In [91]:
new_books = books_users_ratings[['User-ID','unique_isbn',"Book-Rating"]]

In [92]:
new_books.to_csv("Ratings-New.csv", index= False)

In [102]:
from sklearn.model_selection import train_test_split


In [103]:
train, test = train_test_split(new_books, test_size=0.2)


In [105]:
train.to_csv("Train-New.csv", index= False)

In [106]:
test.to_csv("Test-New.csv", index= False)

In [93]:
filter_books = new_books['unique_isbn'].value_counts() > 3
filter_books = filter_books[filter_books].index.tolist()

In [94]:
filter_users = new_books['User-ID'].value_counts() > 3
filter_users = filter_users[filter_users].index.tolist()

In [96]:
print(f'Original shape: {new_books.shape}')
df = new_books[(new_books['unique_isbn'].isin(filter_books)) & (new_books['User-ID'].isin(filter_users))]
print(f'New shape: {df.shape}')

Original shape: (383842, 3)
New shape: (182554, 3)


In [97]:
df.head(6)

,User-ID,unique_isbn,Book-Rating
16,276747,0060517794,9
19,276747,0671537458,9
20,276747,0679776818,8
120,276813,8426449476,8
133,276822,0060096195,10
134,276822,0141310340,9


In [10]:
df.to_csv("Book-Ratings.csv", index = False)

In [99]:
uni_user = df['User-ID'].nunique(dropna=True)
uni_movie = df['unique_isbn'].nunique(dropna=True)
print("number of user")
print(uni_user)
print("number of book")
print(uni_movie)

number of user
15181
number of book
18276


In [100]:
## create item user matrix
total_ratings = uni_user*uni_movie
rating_present = df.shape[0]

rating_not_provide = total_ratings - rating_present


print("number of rating ")
print(total_ratings)
print("number ratings not provide:")
print(rating_not_provide)
## sparsity of matrix user-item
print("sparsity of matrix user-item")
print(rating_not_provide/total_ratings)

number of rating 
277447956
number ratings not provide:
277265402
sparsity of matrix user-item
0.9993420243470815


In [101]:
rating_cnt = pd.DataFrame(df.groupby("Book-Rating").size(), columns=['count'])
rating_cnt

,count
Book-Rating,
1,608
2,1011
3,2228
4,3262
5,18539
6,14203
7,30904
8,44711
9,31741
